In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('Stan_fliter_data.csv')
print(data.shape)
data.head()

In [ ]:
def view_ratio(X,y):
    '''
    查看正负类别比例
    '''
    n_sample = y.shape[0]
    n_pos_sample = y[y==0].shape[0]
    n_neg_sample = y[y==1].shape[0]
    print('样本个数：{}：正样本占{:.2%}：负样本占{:.2%}'.format(n_sample,n_pos_sample/n_sample,n_neg_sample/n_sample))
    print('特征维数:',X.shape[1])

In [ ]:
def up_sampling(X,y):
    '''
    上采样函数，运用SMOTE
    '''
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(random_state=42)
    X,y=sm.fit_sample(X,y)
    return X,y

In [ ]:
X=data.iloc[:,0:9]
y=data.fake
view_ratio(X,y)

In [ ]:
X,y=up_sampling(X,y)
view_ratio(X,y)

In [ ]:
def svm_grid_search(x_train,y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import classification_report
    from sklearn.svm import SVC
    model = SVC(kernel='rbf',probability=True)
    param_grid={'C':[1e-3,1e-4,1e-1,1,10,100,1000],
                'gamma':[0.001,0.0001]}
#     scores = ['recall','precision']
    grid_search = GridSearchCV(model,param_grid=param_grid,cv=5,scoring='recall',verbose=1)
    grid_search.fit(x_train,y_train)
    best_params=grid_search.best_estimator_.get_params()
    for para,val in list(best_params.items()):
        print(para,val)
    model = SVC(kernel='rbf',C=best_params['C'],gamma=best_params['gamma'],probability=True)
    model.fit(x_train,y_train)
    return model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
clf=svm_grid_search(X_train,y_train)
y_true,y_pred = y_test,clf.predict(X_test)
print(classification_report(y_true,y_pred))

In [ ]:
file_test='data/FT_Camp_5/X_test.csv'
test_data=pd.read_csv(file_test)
stay_col=['CA_ratio','Quick_ratio','TA_TO_ratio','Curr_ratio','OtherRec_to_Cur','Pre_to_Cur','Neg_Dednp_times','Monetary_to_Cur','CIP_ratio']
d_test=test_data[stay_col]
d_test.head()

In [ ]:
# 归一化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# d_test=pd.DataFrame(sc.fit_transform(d_test),columns=stay_col)
# d_test.head()
d_test=sc.fit_transform(d_test)

In [ ]:
test_data['fake']=clf.predict(d_test)

In [ ]:
test_data[['stockcode','fake']].to_csv('simple_SVM.csv',sep=',',header=True,index=False,encoding='UTF-8')
test_data[['stockcode','fake']].head()